## MODEL TRAINING


#### 1. Importing data and required packages

In [65]:
import numpy as numpy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

#### 2. Importing CSV data as Pandas Dataframe

In [66]:
df = pd.read_csv('data/stud.csv')

In [67]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


#### 3. Spliting the data into independent variable(X) and dependent variable(Y)

In [68]:
X = df.drop(columns=['math_score'], axis=1)
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


Identifing different categories of features

In [69]:
print("Categories in 'gender' variable:     ", end =" ")
print(df['gender'].unique())

print("Categories in 'race_ethnicity' variable:     ", end=" ")
print(df['race_ethnicity'].unique())

print("Categories in 'parental_level_of_education' variable:    ", end=" ")
print(df['parental_level_of_education'].unique())

print("Categories in 'lunch' variable:      ", end =" ")
print(df['lunch'].unique())

print("Categories in 'test_preparation_course' variable:    ", end=" ")
print(df['test_preparation_course'].unique())

Categories in 'gender' variable:      ['female' 'male']
Categories in 'race_ethnicity' variable:      ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in 'parental_level_of_education' variable:     ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in 'lunch' variable:       ['standard' 'free/reduced']
Categories in 'test_preparation_course' variable:     ['none' 'completed']


In [70]:
Y = df['math_score']

Y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

#### 4. Perfroming One Hot Encoder and Standard Scaler transformation Simultaneously using Column transformation


In [71]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

ss_transfromer = StandardScaler()
ohe_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", ohe_transformer, cat_features),
        ("StandardScaler", ss_transfromer, num_features)
    ]
)

In [72]:
X = preprocessor.fit_transform(X)


In [73]:
X.shape

(1000, 19)

#### 5. Spliting the dataset into training and testing set

In [74]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=15)
X_train.shape, X_test.shape

((800, 19), (200, 19))

#### 6. Creating a Evaluate Function to return the performance after the model has been trained

In [75]:
def evaluate_model(true_value, predicted_value):
    mae = mean_absolute_error(true_value, predicted_value)
    mse = mean_squared_error(true_value, predicted_value)
    rmse = numpy.sqrt(mse)
    r2_square = r2_score(true_value, predicted_value)

    return mae, rmse, r2_square

#### 7. Training Models to check their perfromance

In [ ]:
models = {
    "Linear Regression" : LinearRegression(),
    "Lasso" :  
}